As we have noticed that each of the patient has almost 200+ slices of CT-Scan. While a single CT-Scan slice can carry a lot of information, however, using a single CT-Scan to identify any disease or condition undermines the true purpose of having a CT-Scan. With this view in mind, I intend to sort the slices in the correct order and then producing a 3D volumetric data so that it is possible to get a first hand look at the volumetric information. 

Please upvote the notebook if you think it helped  you somehow.. :) 

In [ ]:
%matplotlib inline

import numpy as np, pandas as pd
import pydicom, imageio, os
import matplotlib.pyplot as plt
from IPython import display
from IPython.display import HTML
from glob import glob
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
import scipy.ndimage
from skimage import morphology
from skimage import measure
from skimage.transform import resize
from sklearn.cluster import KMeans
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.figure_factory as ff
from plotly.graph_objs import *
init_notebook_mode(connected=True) 

At first let's list all the patient IDs so that we can perform the same operation for each of the patient and save the result. 

In [ ]:
train_df = pd.read_csv('../input/rsna-str-pulmonary-embolism-detection/train.csv')
test_df = pd.read_csv('../input/rsna-str-pulmonary-embolism-detection/test.csv')


patient_id_list = os.listdir('../input/rsna-str-pulmonary-embolism-detection/train/')
print(f'Total number of patient(experiment): {len(patient_id_list)}\nFirst 5 patient IDs')
patient_id_list[:5]

Now I will scan all the files of a single patient(experiment). We will just produce a list of file-paths of that particular experiment. Here I have chosen id 6900 because it has both some positive slice and negative slices for Pulmonary Embolism (PE). 

In [ ]:
index = patient_id_list.index('6897fa9de148')
patient_id = patient_id_list[index]
patient_folder = f'../input/rsna-str-pulmonary-embolism-detection/train/{patient_id}/'
patient_image_paths = glob(patient_folder + '/*/*.dcm')
[neg, pos] = train_df[train_df.StudyInstanceUID == patient_id].pe_present_on_image.value_counts().values


# Print out the first 5 file names to verify we're in the right folder.
print (f'Total of {len(patient_image_paths)} DICOM images.' )
print(f'The experiment {patient_id} has {pos} positive and {neg} negative examples for pe_present_on_image\nFirst 5 filenames:')

patient_image_paths[:5]

Now I know that the experiment has both positive and negative samples. So it will be easier to visualize the image plotted. 


Let's make some utility functions to deal with the dicom imges. If you have problem understanding you can have a look at my other notebooks on this same competition. 


In [ ]:
patient_image_paths[0][-16:-4]

In [ ]:
def load_slice(paths):
    slices = [pydicom.read_file(path) for path in paths]
#     labels = [train_df[train_df.SOPInstanceUID == path[-16:-4]].pe_present_on_image.values for path in patient_image_paths]
#     labels = np.array(labels).squeeze()
    slices.sort(key = lambda x: int(x.InstanceNumber), reverse = False)
#     labels.sort(key = lambda x: int(x.InstanceNumber), reverse = False)
    
    try:
        slice_thickness = np.abs(slices[0].ImagePositionPatient[2] - slices[1].ImagePositionPatient[2])
    except:
        slice_thickness = np.abs(slices[0].SliceLocation - slices[1].SliceLocation)    
    for s in slices:
        s.SliceThickness = slice_thickness
        
    return slices

def transform_to_hu(slices):
    images = np.stack([file.pixel_array for file in slices])
    images = images.astype(np.int16)

    # convert ouside pixel-values to air:
    # I'm using <= -1000 to be sure that other defaults are captured as well
    images[images <= -1000] = 0
    
    # convert to HU
    for n in range(len(slices)):    
        intercept = slices[n].RescaleIntercept
        slope = slices[n].RescaleSlope
        if slope != 1:
            images[n] = slope * images[n].astype(np.float64)
            images[n] = images[n].astype(np.int16)      
        images[n] += np.int16(intercept)
    return np.array(images, dtype=np.int16)

In [ ]:
stacked_dicoms = load_slice(patient_image_paths)
stacked_patient_pixels = transform_to_hu(stacked_dicoms)

def sample_stack(stack, rows=6, cols=6, start_with=10, show_every=3):
    fig,ax = plt.subplots(rows,cols,figsize=[20,22])
    for i in range(rows*cols):
        ind = start_with + i*show_every
        ax[int(i/rows),int(i % rows)].set_title(f'slice {ind}')
        ax[int(i/rows),int(i % rows)].imshow(stack[ind],cmap='gray')
        ax[int(i/rows),int(i % rows)].axis('off')
    plt.show()

     
sample_stack(stacked_patient_pixels, show_every = 3)

Now let us have a look at the images with continuous volumetric information. 

In [ ]:
imageio.mimsave(f'./{patient_id}.gif', stacked_patient_pixels, duration=0.1)
display.Image(f'./{patient_id}.gif', format='png')

## Resampling
Although we have each individual slices, it is not immediately clear how thick each slice is. Fortunately, this is in the DICOM header.

In [ ]:
print(f'Slice Thickness: {stacked_dicoms[0].SliceThickness}')
print(f'Pixel Spacing (row, col): ({stacked_dicoms[0].PixelSpacing[0]}, {stacked_dicoms[0].PixelSpacing[1]})')

This means we have 2.0 mm slices, and each voxel represents 0.7 mm.

Because a CT slice is typically reconstructed at 512 x 512 voxels, each slice represents approximately 370 mm of data in length and width.

Using the metadata from the DICOM we can figure out the size of each voxel as the slice thickness. In order to display the CT in 3D isometric form (which we will do below), and also to compare between different scans, it would be useful to ensure that each slice is resampled in 1x1x1 mm pixels and slices.

In [ ]:
np.array([float(stacked_dicoms[0].SliceThickness), float(stacked_dicoms[0].PixelSpacing[0]), float(stacked_dicoms[0].PixelSpacing[0])])

In [ ]:
def resample(image, scan, new_spacing=[1,1,1]):
    # Determine current pixel spacing
#     spacing = map(float, ([scan[0].SliceThickness] + scan[0].PixelSpacing))
    spacing = np.array([float(scan[0].SliceThickness), 
                        float(scan[0].PixelSpacing[0]), 
                        float(scan[0].PixelSpacing[0])])


    resize_factor = spacing / new_spacing
    new_real_shape = image.shape * resize_factor
    new_shape = np.round(new_real_shape)
    real_resize_factor = new_shape / image.shape
    new_spacing = spacing / real_resize_factor
    
    image = scipy.ndimage.interpolation.zoom(image, real_resize_factor)
    
    return image, new_spacing

print(f'Shape before resampling: {stacked_patient_pixels.shape}')
imgs_after_resamp, spacing = resample(stacked_patient_pixels, stacked_dicoms, [1,1,1])
print(f'Shape after resampling: {imgs_after_resamp.shape}')

Let us make a mask that will remvoe the upper and lower part of the slices which are mostly air. 

In [ ]:
def air_removal_mask(dilation):
    labels = measure.label(dilation)
    label_vals = np.unique(labels)
    if labels[0,0] == labels[-1, -1]:
        upper_cut = (labels==labels[0,0])
        mask = np.abs(upper_cut*1 -1) 
    else:
        upper_cut = (labels == labels[0,0])
        lower_cut = (labels == labels[-1,-1])
        mask = np.abs((upper_cut + lower_cut )*1 -1)      
    return mask

Now let us make another mask that will only the lung part of the slices. As a result it will be better for the ML models to concentrate on what's important. 

In [ ]:
#Standardize the pixel values
def make_lungmask(img, display=False):
    row_size= img.shape[0]
    col_size = img.shape[1]
    
    mean = np.mean(img)
    std = np.std(img)
    img = img-mean
    img = img/std
    # Find the average pixel value near the lungs
    # to renormalize washed out images
    middle = img[int(col_size/5):int(col_size/5*4),int(row_size/5):int(row_size/5*4)] 
    mean = np.mean(middle)  
    max = np.max(img)
    min = np.min(img)
    # To improve threshold finding, I'm moving the 
    # underflow and overflow on the pixel spectrum
    img[img==max]=mean
    img[img==min]=mean
    #
    # Using Kmeans to separate foreground (soft tissue / bone) and background (lung/air)
    #
    kmeans = KMeans(n_clusters=2).fit(np.reshape(middle,[np.prod(middle.shape),1]))
    centers = sorted(kmeans.cluster_centers_.flatten())
    threshold = np.mean(centers)
    thresh_img = np.where(img<threshold,1.0,0.0)  # threshold the image

    # First erode away the finer elements, then dilate to include some of the pixels surrounding the lung.  
    # We don't want to accidentally clip the lung.

    eroded = morphology.erosion(thresh_img,np.ones([3,3]))
    dilation = morphology.dilation(eroded,np.ones([8,8]))

    labels = measure.label(dilation) # Different labels are displayed in different colors
    label_vals = np.unique(labels)
    regions = measure.regionprops(labels)
    good_labels = []
    for prop in regions:
        B = prop.bbox
        if B[2]-B[0]<row_size/10*9 and B[3]-B[1]<col_size/10*9 and B[0]>row_size/5 and B[2]<col_size/5*4:
            good_labels.append(prop.label)
    mask = np.ndarray([row_size,col_size],dtype=np.int8)
    mask[:] = 0

    #
    #  After just the lungs are left, we do another large dilation
    #  in order to fill in and out the lung mask 
    #
    for N in good_labels:
        mask = mask + np.where(labels==N,1,0)
    mask = morphology.dilation(mask,np.ones([10,10])) # one last dilation [10,10]
    
    mask = dilation.astype('int16')*air_removal_mask(dilation)
    
    if (display):
        fig, ax = plt.subplots(3, 2, figsize=[12, 12])
        ax[0, 0].set_title("Original")
        ax[0, 0].imshow(img, cmap='gray')
        ax[0, 0].axis('off')
        
        ax[0, 1].set_title("Threshold")
        ax[0, 1].imshow(thresh_img, cmap='gray')
        ax[0, 1].axis('off')
        
        ax[1, 0].set_title("After Erosion and Dilation")
        ax[1, 0].imshow(dilation, cmap='gray')
        ax[1, 0].axis('off')
        
        ax[1, 1].set_title("Color Labels")
        ax[1, 1].imshow(labels)
        ax[1, 1].axis('off')
        
        ax[2, 0].set_title("Final Mask")
        ax[2, 0].imshow(mask, cmap='gray')
        ax[2, 0].axis('off')
        
        ax[2, 1].set_title("Apply Mask on Original")
        ax[2, 1].imshow(mask*img, cmap='gray')
        ax[2, 1].axis('off')
        
        plt.show()
    return mask*img


Let's look at what we have done. 

In [ ]:
img = imgs_after_resamp[120]
output = make_lungmask(img, display=True)

In [ ]:
from tqdm import tqdm
masked_lung = []

for img in tqdm(imgs_after_resamp):
    masked_lung.append(make_lungmask(img))

sample_stack(masked_lung, show_every=6)

In [ ]:
imageio.mimsave(f'segmented{patient_id}.gif', masked_lung, duration=0.1)
display.Image(f'segmented{patient_id}.gif', format='png')

## Saving to npy format
This is a good time to save the new data set to disk so we don't have to reprocess the stack every time. And this is not final. I will make a utility function from this and from that I will convert the whole dataset into this format. 

In [ ]:
np.save(f'resampled_masked_lung_{patient_id}.npy', masked_lung)
np.save(f'stacked_slices_{patient_id}.npy', stacked_patient_pixels)

![](https://www.clipartmax.com/png/middle/265-2655834_work-in-progress-icon.png)





### In the meantime check out my other ongoing works in this same competition: 
💥 [RSNA-STR Pulmonary Embolism [Dummy Sub]](https://www.kaggle.com/redwankarimsony/rsna-str-pulmonary-embolism-dummy-sub)<br>
💥 [CT-Scans, DICOM files, Windowing Explained](https://www.kaggle.com/redwankarimsony/ct-scans-dicom-files-windowing-explained)<br>
💥 [RSNA-STR-PE [Gradient & Sigmoid Windowing]](https://www.kaggle.com/redwankarimsony/rsna-str-pe-gradient-sigmoid-windowing)<br>
💥 [RSNA-STR [✔️3D Stacking ✔️3D Plot ✔️Segmentation]](https://www.kaggle.com/redwankarimsony/rsna-str-3d-stacking-3d-plot-segmentation/edit/run/42517982)<br>
💥 [RSNA-STR [DICOM 👉 GIF 👉 npy]](https://www.kaggle.com/redwankarimsony/rsna-str-dicom-gif-npy)<br>
💥 [RSNA-STR Pulmonary Embolism [EDA]](https://www.kaggle.com/redwankarimsony/rsna-str-pulmonary-embolism-eda)<br>

